In [168]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [169]:
import pandas as pd
tt_test = pd.read_csv("../input/titanic/test.csv")
tt_train = pd.read_csv("../input/titanic/train.csv")

In [170]:
tt_test.info()

In [171]:
tt_train.info()

To combine sets

In [172]:
tt_all = pd.concat([tt_test,tt_train])

In [173]:
tt_all.info()

To check null values


In [174]:
tt_test.isnull().any()

In [175]:
tt_test.isnull().any()

In [176]:
tt_all.isnull().any()

### Age-Fare-Cabin-Embarked-Survived

To deal with missing values:

In [177]:
tt_train.isnull().sum()

In [178]:
tt_test.isnull().sum()

In [179]:
tt_all.isnull().sum()

To examine the relationship between variables

In [180]:
import seaborn as sns 
sns.pairplot(data=tt_train, height=2,
             palette = "Set2",
             plot_kws = {'alpha': 0.4, 's':50, 'edgecolor':'k'},
             markers=["D", "s"], corner = True);

Any linearity?

In [181]:
import matplotlib.pyplot as plt
corr = tt_train.corr()
mask = np.triu(np.ones_like(corr, dtype = bool))
f, ax = plt.subplots(figsize=(9,5.5))
cmap = sns.color_palette("coolwarm", as_cmap=True)
sns. heatmap(data=corr, mask=mask, vmax=1, cmap=cmap, center=0, square= True, linewidths=.5, cbar_kws={"shrink":.9} )

## survived and fare. Naturally, Parch and SibSp are relatively stronger (positive), Pclass and fare are relatively stronger (negative) 

In [182]:
corr.style.background_gradient(cmap="coolwarm")

In [183]:
corrSur = tt_train.corrwith(tt_train["Survived"], method = "spearman").drop("Survived")
plt.figure(figsize=(7,5))
corrbar = sns.barplot(x=corrSur.index, y=corrSur.values, palette="viridis", saturation=0.9)
corrbar.set_xticklabels(corrSur.index, fontsize=15);

Age-Fare-Cabin-Embarked-Survived

For age:

In [184]:
# from sklearn.impute import KNNImputer
# cols= ["Age"]
# X = tt_train[cols]
# impute_knn = KNNImputer(n_neighbors=2)
# impute_knn.fit_transform(X);

In [185]:
tt_test.isnull().any()

In [222]:
tt_train['Age'] = tt_train['Age'].replace(np.nan, 0)
tt_train['Cabin'] = tt_train['Cabin'].replace(np.nan, "C0")
tt_train['Embarked'] = tt_train['Embarked'].replace(np.nan, "Em")
tt_test['Fare'] = tt_test['Fare'].replace(np.nan, 0)
tt_test['Age'] = tt_test['Fare'].replace(np.nan, 0)
tt_test['Cabin'] = tt_test['Fare'].replace(np.nan, 0)


In [223]:
tt_train.isnull().any()

For cabin:

In [188]:
tt_train['Cabin'].value_counts()

In [189]:
tt_train

In [190]:
from sklearn.impute import SimpleImputer
cols = ["Embarked"]
X =tt_train[cols]
imputer = SimpleImputer(strategy='most_frequent', fill_value='missing')
tt_train["Embarked"] = imputer.fit_transform(X)



In [191]:
tt_train["Cabin"].value_counts()

In [192]:
tt_train["FareBoost"] = tt_train["Fare"] ** 22

In [193]:
tt_train.info()

In [194]:
tt_train.columns.values.tolist()


One Hot Encode

In [195]:
tt_train.info()

In [196]:
tt_train


In [214]:
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.compose import make_column_transformer, ColumnTransformer

##X = tt_train.select_dtypes(include=[object])
##X.head(3)
#transformer = ColumnTransformer(transformers=[('cat', OneHotEncoder(), [3, 4, 8, 10, 11])], remainder='passthrough')
from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder()
tt_train["Sex"] = ord_enc.fit_transform(tt_train[["Sex"]])
tt_train["Ticket"] = ord_enc.fit_transform(tt_train[["Ticket"]])
tt_train["Cabin"] = ord_enc.fit_transform(tt_train[["Cabin"]])
tt_train["Embarked"] = ord_enc.fit_transform(tt_train[["Embarked"]])
tt_test["Sex"] = ord_enc.fit_transform(tt_test[["Sex"]])
tt_test["Ticket"] = ord_enc.fit_transform(tt_test[["Ticket"]])
tt_test["Cabin"] = ord_enc.fit_transform(tt_test[["Cabin"]])
tt_test["Embarked"] = ord_enc.fit_transform(tt_test[["Embarked"]])

In [215]:
tt_test

In [212]:
tt_test

In [225]:
#ctrl + '/' comment all shortcut 
f_all = [
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']


X_train = tt_train[f_all].values
y_train = tt_train["Survived"].values
X_test = tt_test[f_all].values


In [226]:
tt_train.isna().sum().sum()

In [227]:
tt_train.isnull().any()

## Hyperparameters tuning

In [204]:
X_train.shape

In [205]:
y_train.shape

In [206]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier()

print(rf.get_params())

params_rf = {"n_estimators" :[100, 150, 200],
             "max_depth" :  [3, 5, 10],
             "max_features" :[0.2, 0.4, 0.6, 0.8],
             "random_state" : [4]}

grid_rf = GridSearchCV(estimator=rf, param_grid=params_rf, scoring="accuracy", cv=10)
grid_rf.fit(X=X_train, y=y_train);

In [207]:
print("Best: %f using %s" % (grid_rf.best_score_, grid_rf.best_params_))

In [262]:
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier(n_estimators=10, max_depth=3, max_features=0.6, random_state=4)
rf.fit(X=X_train, y=y_train)

#Ve modeli test edelim.
y_pred0 = rf.predict(X=X_train)
y_pred1 = rf.predict(X=X_test)

tt_test["Survived"] = y_pred1.tolist()


print(accuracy_score(y_true=y_train, y_pred=y_pred0))



In [263]:
tt_test

In [267]:
preds = tt_test[['PassengerId','Survived']].copy()

In [268]:
preds

In [269]:
preds.to_csv('Predictions.csv',index=False)